In [376]:
## Load packages and data
library(tableone)
library(MatchIt)
library(pROC)
library(ROCR)
library(ggplot2)
library(epitools)
library(dplyr)
#library(gtools)

####loading KKH ICU data
kkh_data <- read.csv("pards_data_0401.csv")
#kkh_data <- read.csv("data_0514.csv")
df<-data.frame(kkh_data)
kkh<-df[!is.na(df$d2_oi),]
kkh <- kkh[which(!((kkh$hfv==1)&(kkh$HFV_verify==0))), ]

####check column names
#colnames(kkh, do.NULL = FALSE)

####select features
ID<-kkh$StudyID
female<-as.numeric(kkh$Gender=='Female')
age<-kkh$Age
pim2<-kkh$pim2
pelod<-kkh$pelod
p_bloodculture<-as.numeric(kkh$Positive_blood_culture=='Yes')
early_HFV<-kkh$early_HFV
#comorbidity
comorbidity<-kkh$comorbidity
#risk factor of ARDS
rf_pneumonia<-as.numeric(kkh$PNEUMONIA=='Checked')
rf_sepsis<-as.numeric(kkh$SEPSIS=='Checked')
rf_aspiration<-as.numeric(kkh$ASPIRATION=='Checked')
rf_transfusion<-as.numeric(kkh$TRANSFUSION=='Checked')
rf_trauma<-as.numeric(kkh$TRAUMA=='Checked')
rf_drowning<-as.numeric(kkh$DROWNING=='Checked')

#other useful features
MODS<-as.numeric(kkh$MODS=='Yes')
OI_category<-kkh$OI_category
treatment<-as.numeric(kkh$Received_HFV=='Yes')
died<-kkh$died_in_ICU

mortality_100day<-kkh$mortality_100day
ventilation_free_day<-kkh$ventilation_free_day
picu_free_days<-kkh$picu_free_days

####new dataset
mydata<-cbind(ID,female,age,pim2,pelod,p_bloodculture,comorbidity,rf_pneumonia,rf_sepsis,rf_aspiration,rf_transfusion
              ,rf_trauma
              ,rf_drowning,MODS,OI_category
              ,treatment,died)

mydata_outcome<-cbind(ID,female,age,pim2,pelod,p_bloodculture,comorbidity,rf_pneumonia,rf_sepsis,rf_aspiration,rf_transfusion
              ,rf_trauma,rf_drowning,MODS,OI_category
              ,treatment,died,mortality_100day,ventilation_free_day,picu_free_days)
mydata_outcome<-data.frame(mydata_outcome)
mydata<-data.frame(mydata)
xvars<-c("female","age","pim2","pelod","p_bloodculture","comorbidity","rf_pneumonia"
         ,"rf_sepsis","rf_aspiration" , "rf_transfusion", "rf_trauma", "rf_drowning","MODS","OI_category"
         )
#re-level 
mydata$OI_category<-as.factor(mydata$OI_category)
mydata$OI_category<-C(mydata$OI_category, contr.treatment, base=3)
#fit a propensity score model with logistic regression
psmodel <-glm(treatment~female+age+pim2+pelod+p_bloodculture+comorbidity+rf_pneumonia+rf_sepsis
                 +rf_aspiration+rf_transfusion+rf_trauma+rf_drowning+MODS+OI_category
                 , data = mydata,family=binomial(link="logit"))
#show coefficients etc
summary(psmodel)

#create propensity score
pscore<-psmodel$fitted.values
#pscore<-fitted(psmodel) 

label<-mydata$treatment
perf<- ROCR::prediction(pscore, label)
psm_auc<-ROCR::performance(ROCR::prediction(pscore, label), "auc")@y.values[1]
print(psm_auc)
#roc.perf = performance(perf, measure = "tpr", x.measure = "fpr")
#plot(roc.perf,col="#1c61b6",percent=TRUE)
#abline(a=0, b= 1)




Call:
glm(formula = treatment ~ female + age + pim2 + pelod + p_bloodculture + 
    comorbidity + rf_pneumonia + rf_sepsis + rf_aspiration + 
    rf_transfusion + rf_drowning + MODS + OI_category, family = binomial(link = "logit"), 
    data = mydata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9023  -0.8269  -0.4074   0.8808   2.7528  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -2.997896   0.818256  -3.664 0.000249 ***
female          0.392725   0.282944   1.388 0.165137    
age            -0.007870   0.030752  -0.256 0.798008    
pim2            0.005246   0.008290   0.633 0.526812    
pelod           0.026776   0.017062   1.569 0.116572    
p_bloodculture  0.431948   0.406125   1.064 0.287517    
comorbidity     0.093346   0.289559   0.322 0.747171    
rf_pneumonia    0.509078   0.445919   1.142 0.253605    
rf_sepsis      -0.725136   0.366442  -1.979 0.047832 *  
rf_aspiration  -0.060818   0.692903  -0.088 0.93

[[1]]
[1] 0.812311



In [392]:
#########################################
#look at a table 1
#mydata before matching
table1_unmatched<- CreateTableOne(vars=xvars, strata="treatment", data = mydata)
## include standardized mean difference (SMD)
print(table1_unmatched, smd=TRUE)

## Using the Matchit for propensity score, nearest neighbor matching
mydata_wth_pscore<-data.frame(cbind(mydata,pscore))
m.out <- matchit(treatment~female+age+pim2+pelod+p_bloodculture+comorbidity+rf_pneumonia+rf_sepsis
                 +rf_aspiration+rf_transfusion+rf_trauma+rf_drowning+MODS+OI_category, data = mydata_wth_pscore,              
                 method = "nearest", distance=mydata_wth_pscore$pscore,caliper=0.1
                 ,ratio=1)
summary(m.out)

#propensity score plots
plot(m.out, type="jitter")
plot(m.out, type="hist")

#MatchIt Table1
table1_matched<- CreateTableOne(vars=xvars, strata="treatment", data = match.data(m.out, "all"))
## include standardized mean difference (SMD)
print(table1_matched, smd=TRUE)

#calculate odds ratio for all covariates in the psmodel
exp(cbind(Odds_and_OR=coef(psmodel), confint(psmodel)))


Warning message in ModuleReturnVarsExist(vars, data):
“The data frame does not have: rf_trauma  Dropped”Warning message in ModuleReturnVarsExist(vars, data):
“The data frame does not have: rf_trauma  Dropped”

                            Stratified by treatment
                             0             1             p      test SMD   
  n                             82            82                           
  female (mean (sd))          0.45 (0.50)   0.45 (0.50)   1.000      <0.001
  age (mean (sd))             4.26 (4.62)   4.32 (4.76)   0.931       0.014
  pim2 (mean (sd))           17.73 (21.88) 16.55 (17.88)  0.707       0.059
  pelod (mean (sd))          11.01 (10.64) 10.83 (9.69)   0.909       0.018
  p_bloodculture (mean (sd))  0.20 (0.40)   0.18 (0.39)   0.843       0.031
  comorbidity (mean (sd))     0.55 (0.50)   0.55 (0.50)   1.000      <0.001
  rf_pneumonia (mean (sd))    0.82 (0.39)   0.83 (0.38)   0.839       0.032
  rf_sepsis (mean (sd))       0.34 (0.48)   0.32 (0.47)   0.742       0.052
  rf_aspiration (mean (sd))   0.05 (0.22)   0.04 (0.19)   0.701       0.060
  rf_transfusion (mean (sd))  0.01 (0.11)   0.01 (0.11)   1.000      <0.001
  rf_drowning (mean (sd))     0.02 (

In [393]:
###################################
#outcome analysis
y_treatment<-match.data(m.out)$died[match.data(m.out, "all")$treatment==1] 
y_control<-match.data(m.out)$died[match.data(m.out, "all")$treatment==0] 

table(y_treatment,y_control)
#McNemar test for PICU mortality
mcnemar.test(y_treatment, y_control)

########## calculate Odd's ratio here!
a<-length(y_treatment[y_treatment==1])
b<-length(y_control[y_control==1])
c<-length(y_treatment[y_treatment==0])
d<-length(y_control[y_control==0])

oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))

           y_control
y_treatment  0  1
          0 35 15
          1 19 13


	McNemar's Chi-squared test with continuity correction

data:  y_treatment and y_control
McNemar's chi-squared = 0.26471, df = 1, p-value = 0.6069


,Disease1,Disease2,Total
Exposed1,32,50,82
Exposed2,28,54,82
Total,60,104,164
,estimate,lower,upper
Exposed1,1.000000,NA,NA
Exposed2,1.232028,0.6502002,2.344894
,midp.exact,fisher.exact,chi.square
Exposed1,NA,NA,NA
Exposed2,0.5223135,0.6269034,0.5166813


In [394]:
matchedID_hfv<-match.data(m.out, "treat")$ID
matchedID_non_hfv<-match.data(m.out, "control")$ID
pairs<-cbind(matchedID_hfv,matchedID_non_hfv)
#write.csv(pairs, file = "pair_ID.csv")

In [397]:
pairs <- read.csv("pair_ID.csv")
#head(pairs)
pairs<-data.frame(pairs)
data<-mydata_outcome[(mydata_outcome$ID %in% pairs$matchedID_hfv)| (mydata_outcome$ID %in% pairs$matchedID_non_hfv),]
#head(data)
#length(data[,1])

#pairwise difference for continuous data
hfv<-data[data$ID %in% pairs$matchedID_hfv,]
non_hfv<-data[data$ID %in% pairs$matchedID_non_hfv,]

#paired t-test for ventilator free days
diffy<-hfv$ventilation_free_day-non_hfv$ventilation_free_day
t.test(diffy)
#paired t-test for picu free days
diffy1<-hfv$picu_free_days-non_hfv$picu_free_days
t.test(diffy1)

#McNemar test for 100 day mortality
mcnemar.test(hfv$mortality_100day, non_hfv$mortality_100day)



	One Sample t-test

data:  diffy
t = -1.3336, df = 81, p-value = 0.1861
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -4.6496170  0.9179097
sample estimates:
mean of x 
-1.865854 



	One Sample t-test

data:  diffy1
t = -1.9352, df = 81, p-value = 0.05645
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -4.77354950  0.06623243
sample estimates:
mean of x 
-2.353659 



	McNemar's Chi-squared test with continuity correction

data:  hfv$mortality_100day and non_hfv$mortality_100day
McNemar's chi-squared = 0.59259, df = 1, p-value = 0.4414
